## Feature Engineering in Genomics

Feature Engineering is the process of transforming raw data into features/input variables that are easily digested by algorithms. People think that data scientists often spend most of their time testing out various algorithms; however, the majority of performance gains generally come from well-crafted features.

While performing feature engineering, it is critical to keep in mind the question that you are trying to answer. For the purposes of this exercise, we will be using ...genomic data, with an aims to answer the following questions:


In this notebook, we will introduce the following types of feature engineering:
- Feature pruning
- Time-based features (month, year, etc)
- One-hot encoding to create dummy variables
- Extracting features from strings
- Feature scaling
- Data imputation / cleaning

How to transform your genomics data to fit into machine learning models

### Converting DNA Sequence String into NumPy Array

## 1. Ordinal / Label Encoding
this label (ordinary) encoding will encode each base nucleotide as a custom numerical value. Mostly use `'A':0.25, 'C':0.5,'G':0.75, 'T':1.0`, but sometimes may use `A':1, 'C':2,'G':3, 'T':4`, which is not recommended. 

Here is a custom code for label encoding. However, `sklearn` has an inbuilt encoder. 

In [1]:
dna_sequence_string = "ATATATCCCGGGAATTTTCGTAGTTAGGCTGATTTTATTGGCGCGAAAATTTTTT"

In [2]:
dna_dict_per = {'A':0.25, 'C':0.5,'G':0.75, 'T':1.0, 'N':0 }
dna_dict_int = {'A':1, 'C':2,'G':3, 'T':4, 'N':0 }

In [3]:
label_encode = [dna_dict_per[dna] for dna in list(dna_sequence_string)]

In [4]:
label_encode

[0.25,
 1.0,
 0.25,
 1.0,
 0.25,
 1.0,
 0.5,
 0.5,
 0.5,
 0.75,
 0.75,
 0.75,
 0.25,
 0.25,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 0.75,
 1.0,
 0.25,
 0.75,
 1.0,
 1.0,
 0.25,
 0.75,
 0.75,
 0.5,
 1.0,
 0.75,
 0.25,
 1.0,
 1.0,
 1.0,
 1.0,
 0.25,
 1.0,
 1.0,
 0.75,
 0.75,
 0.5,
 0.75,
 0.5,
 0.75,
 0.25,
 0.25,
 0.25,
 0.25,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [5]:
import numpy as np
import re
def string_to_array(seq_string):
    seq_string = seq_string.lower()
    seq_string = re.sub('[^acgt]', 'n', seq_string)
    seq_string = np.array(list(seq_string))
    return seq_string
# create a label encoder with 'acgtn' alphabet
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [6]:
def ordinal_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, lets say n
    return float_encoded

In [7]:
seq_test = 'TTCAGCCAGTG'
ordinal_encoder(string_to_array(seq_test))

array([1.  , 1.  , 0.5 , 0.25, 0.75, 0.5 , 0.5 , 0.25, 0.75, 1.  , 0.75])

## 2. One hot Encoder
For the standard base of nucleotides, the “ACGT” sequence string will be one-hot encoded as [[1. 0. 0. 0.] [0. 1. 0. 0.] [0. 0. 1. 0.] [0. 0. 0. 1.]] using the NumPy array [‘a’ ‘c’ ‘g’ ‘t’].

In [8]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re

class hot_dna:
    def __init__(self,fasta):
        #check for and grab sequence name
        if re.search(">",fasta):
            name = re.split("\n",fasta)[0]
            sequence = re.split("\n",fasta)[1]
        else :
            name = 'unknown_sequence'
            sequence = fasta
  
        #get sequence into an array
        seq_array = array(list(sequence))
    
        #integer encode the sequence
        label_encoder = LabelEncoder()
        integer_encoded_seq = label_encoder.fit_transform(seq_array)
    
        #one hot the sequence
        onehot_encoder = OneHotEncoder(sparse=False)
        #reshape because that's what OneHotEncoder likes
        integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
        onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)

        #add the attributes to self 
        self.name = name
        self.sequence = fasta
        self.integer = integer_encoded_seq
        self.onehot = onehot_encoded_seq

In [9]:
fasta = ">fake_sequence\nATGTGTCGTAGTCGTACG"
my_hottie = hot_dna(fasta)

In [10]:
my_hottie.onehot

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [11]:
from sklearn.preprocessing import OneHotEncoder
def one_hot_encoder(seq_string):
    int_encoded = label_encoder.transform(seq_string)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int)
    int_encoded = int_encoded.reshape(len(int_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(int_encoded)
    onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded

In [12]:
seq_test = 'GAATTCTCGAA'
one_hot_encoder(string_to_array(seq_test))

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

## K-mer counting
Another form of encoding, mostly used in Natural language processing is k-mer counting. Here we can decide on the size of the k-mer and then count them in various sequences, then see how if that can correlate.

We first take the long biological sequence and break it down into k-mer length overlapping “words”. For example, if we use “words” of length 6 (hexamers), “ATGCATGCA” becomes: ‘ATGCAT’, ‘TGCATG’, ‘GCATGC’, ‘CATGCA’. Hence our example sequence is broken down into 4 hexamer words.

In genomics, we refer to these types of manipulations as [“k-mer counting”](https://en.wikipedia.org/wiki/K-mer), or counting the occurrences of each possible k-mer sequence and Python natural language processing tools make it super easy.

In [13]:
def Kmers_funct(seq, k):
    return [seq[x:x+k].lower() for x in range(len(seq) - k + 1)]

In [15]:
mySeq = 'GTGCCCAGGTTCAGTGAGTGACACAGGCAG'
Kmers_funct(mySeq, k=5)

['gtgcc',
 'tgccc',
 'gccca',
 'cccag',
 'ccagg',
 'caggt',
 'aggtt',
 'ggttc',
 'gttca',
 'ttcag',
 'tcagt',
 'cagtg',
 'agtga',
 'gtgag',
 'tgagt',
 'gagtg',
 'agtga',
 'gtgac',
 'tgaca',
 'gacac',
 'acaca',
 'cacag',
 'acagg',
 'caggc',
 'aggca',
 'ggcag']

We can the convert the kmers to words, and then use the natural language processing methods. 

In [17]:
words = Kmers_funct(mySeq, k=6)
joined_sentence = ' '.join(words)
joined_sentence

'gtgccc tgccca gcccag cccagg ccaggt caggtt aggttc ggttca gttcag ttcagt tcagtg cagtga agtgag gtgagt tgagtg gagtga agtgac gtgaca tgacac gacaca acacag cacagg acaggc caggca aggcag'

In [31]:
mySeq1 = 'TCTCACACATGTGCCAATCACTGTCACCCTCTCACACA'
mySeq2 = 'GTGCCCAGGTTCAGTGAGTGACACAGGCAGTCTCACACA'
sentence1 = ' '.join(Kmers_funct(mySeq1, k=6))
sentence2 = ' '.join(Kmers_funct(mySeq2, k=6))

In [32]:
sentence1

'tctcac ctcaca tcacac cacaca acacat cacatg acatgt catgtg atgtgc tgtgcc gtgcca tgccaa gccaat ccaatc caatca aatcac atcact tcactg cactgt actgtc ctgtca tgtcac gtcacc tcaccc caccct accctc ccctct cctctc ctctca tctcac ctcaca tcacac cacaca'

In [33]:
sentence2

'gtgccc tgccca gcccag cccagg ccaggt caggtt aggttc ggttca gttcag ttcagt tcagtg cagtga agtgag gtgagt tgagtg gagtga agtgac gtgaca tgacac gacaca acacag cacagg acaggc caggca aggcag ggcagt gcagtc cagtct agtctc gtctca tctcac ctcaca tcacac cacaca'

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform([joined_sentence, sentence1, sentence2]).toarray()

In [35]:
X

array([[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 2, 1, 1, 0, 2, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1]])

## Feature Scaling

We will not overwrite our dataframe with scaled values because the appropriate scaling technique depends on the algorithm.  These are the three most common feature scaling techniques:
1. Normalization
2. Standardization
3. Log-transformation

### Normalization -- Min-max scaling

- Min-max scaling squashes the features into a [0, 1] range, which can be achieved via the following equation for a single input $i$:

$$x^{[i]}_{\text{norm}} = \frac{x^{[i]} - x_{\text{min}} }{ x_{\text{max}} - x_{\text{min}} }$$

Normalization is the process of rescaling the data from 0-1.  The formula for this approach is:

`X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) X_scaled = X_std * (max - min) + min` 

### Standardization

- Z-score standardization is a useful standardization scheme if we are working with certain optimization methods (e.g., gradient descent). 
- After standardizing a feature, it will have the properties of a standard normal distribution, that is, unit variance and zero mean ($N(\mu=0, \sigma^2=1)$); however, this does not transform a feature from not following a normal distribution to a normal distributed one.
- The formula for standardizing a feature is shown below, for a single data point $x^{[i]}$.

$$x^{[i]}_{\text{std}} = \frac{x^{[i]} - \mu_x }{ \sigma_{x} }$$
